In [ ]:
"""ICSARA (SEIA) — Extracción heurística desde PDF a JSON

Descripción:
Proceso heurístico desarrollado para transformar un ICSARA en formato PDF en un dataset
estructurado, trazable y reutilizable. El sistema no se basa en marcadores formales del
documento, sino que **determina** capítulos, bisagras, preguntas y elementos gráficos a
partir de patrones de layout, tipografía, posición relativa y continuidad entre páginas.

Flujo:
  1. Abrir el PDF una única vez y recorrerlo secuencialmente por página.
  2. Determinar tablas vectoriales y figuras raster mediante heurísticas de layout
     (detección por bounding boxes y características gráficas).
  3. Recortar y exportar cada tabla/figura a PNG con nomenclatura estable:
       p{numero_pregunta}_parte{N}_{tabla|figura}.png
  4. Extraer texto plano excluyendo heurísticamente las áreas ocupadas por tablas y figuras,
     evitando duplicación de contenido y ruido visual.
  5. Determinar capítulos mediante heurísticas tipográficas (romanos en negrita, tamaño de
     fuente, posición) y resolver continuidad entre páginas (cross-page).
  6. Determinar bisagras mediante patrones de layout y semántica superficial, asociándolas
     a las preguntas correspondientes.
  7. Limpiar heurísticamente artefactos no textuales: firmas digitales, encabezados
     repetidos y bisagras residuales.
  8. Consolidar cada pregunta en una estructura consistente con su capítulo, bisagra,
     texto limpio y referencias a tablas/figuras asociadas.

Salidas:
  - preguntas.json
      Dataset estructurado por pregunta:
      {capitulo, bisagra, numero, texto,
       tablas_figuras:[{tipo, parte, png}]}

  - preguntas.txt
      Representación legible y continua del contenido textual.

  - outputs_png/
      Recortes de tablas y figuras con trazabilidad directa a cada pregunta.

  - chapters_hinges.json
      Archivo de apoyo para validación y depuración de las heurísticas de detección.
"""

import os
import re
import json
from bisect import bisect_right
from collections import Counter, defaultdict
from pathlib import Path
import fitz  # pymupdf


# =============================================================================
# CONFIG
# =============================================================================
BASE_DIR = Path(os.getenv("ICSARA_BASE_DIR", str(Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd())))
PDF_PATH = Path(os.getenv("ICSARA_PDF_PATH", str(BASE_DIR / "1766432953_2167380849.pdf")))
OUT_DIR = Path(os.getenv("ICSARA_OUT_DIR", str(BASE_DIR / "salida_icsara")))
SEPARADOR_PREGUNTA = "------------"

# --- Texto ---
PAT_PREGUNTA = re.compile(r"(?m)^\s*(\d{1,4})\.\s+")
FRASES_RUIDO = [
    "Para validar las firmas de este documento",
    "sea.gob.cl/validar", "validar las firmas",
    "https://validador.sea.gob.cl/validar",
    "Firmado Digitalmente", "sellodigital.sea.gob.cl",
    "Razón:", "Razon:",
]
FIRMA_TOKENS = (
    "firmado digitalmente", "sellodigital", "utc",
    "fecha:", "razón", "razon", "lugar:",
)
MIN_RATIO_FRECUENTES = 0.60
YEAR_MIN, YEAR_MAX = 1900, 2100
MONO_DROP_THRESHOLD = 5

RE_CAP_ROM_TIT = re.compile(r"^\s*([IVXLCDM]{1,10})\.\s+(.+?)\s*$", re.IGNORECASE)
RE_CAP_ROM_SOLO = re.compile(r"^\s*([IVXLCDM]{1,10})\.\s*$", re.IGNORECASE)
RE_NUM_SOLO = re.compile(r"^\s*\d{1,4}\.\s*$")

RE_TABLA_PARTES = re.compile(r"(?i)^\s*Tabla\s+XX\.\s*Partes\s+y\s+obras\s+del\s+Proyecto\s*$")
RE_NOMBRE_PARTE = re.compile(r"^\s*\[(Nombre\s+parte/obra\s+.+?)\]\s*$", re.IGNORECASE)
RE_CARACTER = re.compile(r"^\s*\[(Temporal\s+o\s+permanente)\]\s*$", re.IGNORECASE)
RE_FASE = re.compile(r"^\s*\[(Construcción.*?cierre)\]\s*$", re.IGNORECASE)

RE_FIRMA_BLOQUE = re.compile(
    r"(?:Fecha:\s*\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\s+\d{1,2}:\d{2}[:\d.]*\s*(?:UTC\s*[+-]?\d{2}:\d{2})?\s*(?:Lugar:\s*)?)+",
    re.IGNORECASE,
)
RE_FIRMA_COMPLETA = re.compile(r"(?:Firmado\s+Digitalmente\s+por\s+.+?)(?=\n|$)", re.IGNORECASE)
RE_FECHA_PRE_FIRMA = re.compile(r"\d{1,2}\s+de\s+\w+\s+de\s+\d{4}\.\s*$", re.IGNORECASE)

# --- Detección tablas/figuras ---
THIN_MAX = 1.2
MIN_LINE_LEN = 25.0
MIN_HLINES = 6
MIN_VLINES = 4
MERGE_GAP = 12.0
MIN_TABLE_AREA = 15_000.0
MIN_FIG_AREA = 8_000.0
PNG_DPI = 200
PNG_DIRNAME = "outputs_png"

# --- Layout ---
SAME_LINE_Y = 2.5
SAME_LINE_X_GAP = 22.0
Y_GAP_MERGE = 6.0
X_TOL_MERGE = 24.0
MAX_BISAGRA_TO_Q_GAP = 55.0
BOTTOM_PAGE_MARGIN = 120.0
TOP_NEXT_PAGE_SEARCH = 250.0

RE_ROMAN = re.compile(r"^\s*([IVXLCDM]{1,10})\.\s+(.+?)\s*$", re.IGNORECASE)
RE_QSTART = re.compile(r"^\s*(\d{1,4})\.\s+")


# #############################################################################
#  GEOMETRÍA
# #############################################################################
def rect_area(r):
    return max(0.0, (r.x1 - r.x0)) * max(0.0, (r.y1 - r.y0))

def union_rect(a, b):
    return fitz.Rect(min(a.x0, b.x0), min(a.y0, b.y0), max(a.x1, b.x1), max(a.y1, b.y1))

def intersects(a, b):
    return a.intersects(b)

def rect_close(a, b, gap):
    dx = max(0.0, max(a.x0 - b.x1, b.x0 - a.x1))
    dy = max(0.0, max(a.y0 - b.y1, b.y0 - a.y1))
    return dx <= gap and dy <= gap

def merge_rects(rects, gap=MERGE_GAP):
    rects = [fitz.Rect(r) for r in rects]
    out = []
    for r in rects:
        merged = False
        for i in range(len(out)):
            if rect_close(out[i], r, gap):
                out[i] = union_rect(out[i], r)
                merged = True
                break
        if not merged:
            out.append(r)
    changed = True
    while changed:
        changed = False
        new_out = []
        while out:
            r = out.pop()
            merged_any = False
            for i in range(len(out)):
                if rect_close(out[i], r, gap):
                    out[i] = union_rect(out[i], r)
                    merged_any = True
                    changed = True
                    break
            if not merged_any:
                new_out.append(r)
        out = new_out
    return out

def in_any_rect(point_y0, rects):
    """Verifica si una coordenada y0 cae dentro de algún rect."""
    for r in rects:
        if r.y0 <= point_y0 <= r.y1:
            return True
    return False


# #############################################################################
#  DETECCIÓN TABLAS VECTORIALES Y FIGURAS RASTER
# #############################################################################
def extract_table_candidates(page):
    drawings = page.get_drawings()
    h_lines, v_lines = [], []
    for d in drawings:
        for it in d.get("items", []):
            op = it[0]
            if op == "l":
                (x1, y1), (x2, y2) = it[1], it[2]
                dx, dy = abs(x2 - x1), abs(y2 - y1)
                if dx >= MIN_LINE_LEN and dy <= 1.0:
                    h_lines.append(fitz.Rect(min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)))
                elif dy >= MIN_LINE_LEN and dx <= 1.0:
                    v_lines.append(fitz.Rect(min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)))
            elif op == "re":
                r = fitz.Rect(it[1])
                w, h = abs(r.x1 - r.x0), abs(r.y1 - r.y0)
                if h <= THIN_MAX and w >= MIN_LINE_LEN:
                    h_lines.append(r)
                elif w <= THIN_MAX and h >= MIN_LINE_LEN:
                    v_lines.append(r)
    if len(h_lines) < MIN_HLINES or len(v_lines) < MIN_VLINES:
        return []
    all_rects = h_lines + v_lines
    bbox = all_rects[0]
    for r in all_rects[1:]:
        bbox = union_rect(bbox, r)
    if rect_area(bbox) < MIN_TABLE_AREA:
        return []
    merged = merge_rects(all_rects, gap=MERGE_GAP)
    groups = merge_rects(merged, gap=MERGE_GAP * 2)
    return [g for g in groups if rect_area(g) >= MIN_TABLE_AREA]


def extract_raster_figures(page):
    figs = []
    for img in page.get_images(full=True):
        xref = img[0]
        for r in page.get_image_rects(xref):
            rr = fitz.Rect(r)
            if rect_area(rr) >= MIN_FIG_AREA:
                figs.append(rr)
    return merge_rects(figs, gap=10.0)


def save_bbox_screenshot(doc, page_index0, bbox, out_dir, fname, dpi=PNG_DPI):
    page = doc[page_index0]
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, clip=fitz.Rect(bbox), alpha=False)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / fname
    pix.save(out_path.as_posix())
    return out_path.as_posix()


# #############################################################################
#  TEXTO PLANO — EXCLUYENDO ZONAS DE TABLAS/FIGURAS
# #############################################################################
def extract_page_text_excluding_bboxes(page, exclude_rects):
    """
    Extrae texto de la página excluyendo las zonas de tablas/figuras.
    Usa page.get_text("dict") y filtra bloques/líneas cuyos spans
    caigan dentro de algún rect excluido.
    """
    d = page.get_text("dict")
    out_lines = []

    for block in d.get("blocks", []):
        if block.get("type") != 0:
            continue
        for line in block.get("lines", []):
            line_bbox = fitz.Rect(line["bbox"])
            # Si la línea intersecta con algún rect excluido, omitirla
            skip = False
            for er in exclude_rects:
                if intersects(line_bbox, er):
                    skip = True
                    break
            if skip:
                continue
            # Reconstruir texto de la línea
            text = "".join(sp.get("text", "") for sp in line.get("spans", []))
            out_lines.append(text)

    return "\n".join(out_lines)


def normalize_lines_keep_empty(text):
    return [ln.rstrip("\r") for ln in text.replace("\x0c", "\n").split("\n")]


# #############################################################################
#  FILTRO HEADERS/FOOTERS + STITCH
# #############################################################################
def build_frequent_line_filter(pages_lines, min_ratio=MIN_RATIO_FRECUENTES):
    n = len(pages_lines)
    c = Counter()
    for lines in pages_lines:
        for ln in set(ln.strip() for ln in lines if ln.strip()):
            c[ln] += 1
    threshold = max(2, int(n * min_ratio))
    return {ln for ln, k in c.items() if k >= threshold}


def clean_page_lines_keep_empty(lines, frequent_lines):
    out = []
    for ln in lines:
        s = ln.strip()
        if s == "":
            out.append("")
            continue
        if s in frequent_lines:
            continue
        low = s.lower()
        if any(fr.lower() in low for fr in FRASES_RUIDO):
            continue
        if re.fullmatch(r"\d{1,4}", s):
            continue
        out.append(s)
    return out


def stitch_pages(pages_clean_lines):
    texto = ""
    for lines in pages_clean_lines:
        page_text = "\n".join(lines).strip()
        if not page_text:
            continue
        if not texto:
            texto = page_text
            continue
        prev = texto.rstrip()
        if prev.endswith("-"):
            texto = prev[:-1] + page_text.lstrip()
        else:
            texto += "\n" + page_text.lstrip()
    return texto


# #############################################################################
#  NORMALIZACIÓN Y FORMATEO DE TEXTO
# #############################################################################
def normalize_text_preserving_paragraphs(text):
    text = re.sub(r"-\n(?=\w)", "", text)
    text = re.sub(r"\n\s*\n+", "\n<<<PARA>>>\n", text)
    list_start = r"(?:[A-Za-z]\)|\d+\)|\([A-Za-z0-9]+\)|[-•])"
    text = re.sub(rf"\n\s*(?={list_start})", "\n<<<PARA>>>\n", text)
    text = re.sub(r"\n+", " ", text)
    text = text.replace("<<<PARA>>>", "\n")
    text = re.sub(r"[ \t]{2,}", " ", text).strip()
    return text


def looks_table_row_horizontal(line):
    if re.search(r"\s{2,}", line.strip()):
        cols = [c for c in re.split(r"\s{2,}", line.strip()) if c.strip()]
        return len(cols) >= 2
    return False


def split_table_row_horizontal(line):
    return [re.sub(r"\s{2,}", " ", c.strip()) for c in re.split(r"\s{2,}", line.strip()) if c.strip()]


def format_horizontal_table_as_semicolon(lines):
    rows = [split_table_row_horizontal(ln) for ln in lines if ln.strip()]
    if not rows:
        return ""
    mx = max(len(r) for r in rows)
    rows = [r + [""] * (mx - len(r)) for r in rows]
    return "\n".join(";".join(r) for r in rows)


def parse_tabla_partes_obras(lines, start_idx):
    i = start_idx
    if i >= len(lines) or not RE_TABLA_PARTES.match(lines[i].strip()):
        return "", start_idx
    table_title = lines[i].strip()
    i += 1
    while i < len(lines) and not RE_NOMBRE_PARTE.match(lines[i].strip()):
        i += 1
    rows = []
    header = ["Tabla", "Nombre", "Descripción", "Carácter", "Fase"]
    while i < len(lines):
        if RE_TABLA_PARTES.match(lines[i].strip()):
            i += 1
            while i < len(lines) and not RE_NOMBRE_PARTE.match(lines[i].strip()):
                i += 1
            continue
        m_name = RE_NOMBRE_PARTE.match(lines[i].strip())
        if not m_name:
            break
        nombre = f"[{m_name.group(1)}]"
        i += 1
        desc_lines = []
        while i < len(lines) and not RE_CARACTER.match(lines[i].strip()):
            if RE_NOMBRE_PARTE.match(lines[i].strip()):
                break
            desc_lines.append(lines[i])
            i += 1
        descripcion = normalize_text_preserving_paragraphs("\n".join(desc_lines)).strip()
        caracter = ""
        if i < len(lines) and RE_CARACTER.match(lines[i].strip()):
            caracter = "Temporal o permanente"
            i += 1
        fase = ""
        if i < len(lines):
            m_f = RE_FASE.match(lines[i].strip())
            if m_f:
                fase = m_f.group(1)
                i += 1
            elif lines[i].strip().startswith("[") and lines[i].strip().endswith("]"):
                fase = lines[i].strip().strip("[]").strip()
                i += 1
        rows.append([table_title, nombre, descripcion, caracter, fase])
        while i < len(lines) and lines[i].strip() == "":
            i += 1
        if i < len(lines) and RE_NUM_SOLO.match(lines[i]):
            break
    if not rows:
        return "", start_idx
    out_lines = [";".join(header)]
    for r in rows:
        out_lines.append(";".join(r))
    return "\n".join(out_lines), i


def format_question(texto_raw):
    lines = normalize_lines_keep_empty(texto_raw)
    out_parts = []
    buf_text = []

    def flush_text():
        nonlocal buf_text
        if not buf_text:
            return
        block = "\n".join(buf_text).strip()
        if block:
            out_parts.append(normalize_text_preserving_paragraphs(block))
        buf_text = []

    i = 0
    while i < len(lines):
        ln = lines[i]
        if ln.strip() == "":
            buf_text.append("")
            i += 1
            continue
        if RE_TABLA_PARTES.match(ln.strip()):
            flush_text()
            table_csv, new_i = parse_tabla_partes_obras(lines, i)
            if table_csv:
                out_parts.append(table_csv)
                i = new_i
                continue
        if looks_table_row_horizontal(ln):
            flush_text()
            table_lines = []
            while i < len(lines) and lines[i].strip() != "" and looks_table_row_horizontal(lines[i]):
                table_lines.append(lines[i])
                i += 1
            table_txt = format_horizontal_table_as_semicolon(table_lines)
            if table_txt:
                out_parts.append(table_txt)
            continue
        buf_text.append(ln)
        i += 1
    flush_text()
    return "\n".join(p for p in out_parts if p).strip()


# #############################################################################
#  LIMPIEZA POST-FORMATO
# #############################################################################
def clean_firma_digital(texto):
    texto = RE_FIRMA_BLOQUE.sub("", texto)
    texto = RE_FIRMA_COMPLETA.sub("", texto)
    texto = texto.rstrip()
    m = RE_FECHA_PRE_FIRMA.search(texto)
    if m:
        pos = m.start()
        remaining = RE_FECHA_PRE_FIRMA.sub("", texto[pos:]).strip()
        if not remaining:
            texto = texto[:pos].rstrip()
    return re.sub(r"[\s\n]+$", "", texto)


def clean_trailing_hinge(texto, all_hinge_texts):
    if not all_hinge_texts:
        return texto
    ts = texto.rstrip()
    for ht in all_hinge_texts:
        ht = ht.strip()
        if not ht:
            continue
        if ts.endswith(ht):
            ts = ts[:-len(ht)].rstrip()
            break
        if ht.endswith(".") and ts.endswith(ht[:-1]):
            ts = ts[:-(len(ht) - 1)].rstrip()
            break
    return re.sub(r"[\s\n]+$", "", ts)


# #############################################################################
#  PREGUNTAS — DETECCIÓN DESDE TEXTO PLANO
# #############################################################################
def get_line_bounds(text, pos):
    ls = text.rfind("\n", 0, pos) + 1
    le = text.find("\n", pos)
    return ls, (le if le != -1 else len(text))


def next_nonempty_line(text, start_pos):
    i = start_pos
    n = len(text)
    while i < n:
        j = text.find("\n", i)
        if j == -1:
            return text[i:].strip()
        line = text[i:j].strip()
        if line:
            return line
        i = j + 1
    return ""


def is_false_question_start(texto_total, match_start, num_str):
    ls, le = get_line_bounds(texto_total, match_start)
    line = texto_total[ls:le].strip()
    low = line.lower()
    if len(num_str) == 4:
        try:
            n = int(num_str)
            if YEAR_MIN <= n <= YEAR_MAX and any(t in low for t in FIRMA_TOKENS):
                return True
        except: pass
    only_num = bool(re.fullmatch(r"\s*" + re.escape(num_str) + r"\.\s*", line))
    if not only_num:
        return False
    if len(num_str) == 4:
        try:
            n = int(num_str)
            if YEAR_MIN <= n <= YEAR_MAX:
                return True
        except: pass
    if len(num_str) >= 2 and num_str[0] == "0":
        return True
    nxt = next_nonempty_line(texto_total, le + 1)
    if nxt and not re.match(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]", nxt[0]):
        return True
    return False


def apply_monotonic_filter(starts):
    kept = []
    last_num = None
    for ini, num in starts:
        try: n = int(num)
        except: continue
        if last_num is None:
            kept.append((ini, num)); last_num = n; continue
        if n < last_num and (last_num - n) >= MONO_DROP_THRESHOLD:
            continue
        kept.append((ini, num)); last_num = n
    return kept


def extract_questions_from_text(texto_total):
    matches = list(PAT_PREGUNTA.finditer(texto_total))
    starts = []
    for m in matches:
        ini, num = m.start(), m.group(1)
        if not is_false_question_start(texto_total, ini, num):
            starts.append((ini, num))
    starts = apply_monotonic_filter(starts)
    out = []
    for i, (ini, num) in enumerate(starts):
        fin = starts[i + 1][0] if i + 1 < len(starts) else len(texto_total)
        raw_sin = re.sub(rf"^\s*{re.escape(num)}\.\s*", "", texto_total[ini:fin].strip(), count=1)
        out.append({"numero": int(num), "texto": format_question(raw_sin)})
    return out


# #############################################################################
#  LAYOUT — CAPÍTULOS Y BISAGRAS
# #############################################################################
def extract_spans(page):
    d = page.get_text("dict")
    spans = []
    for block in d.get("blocks", []):
        if block.get("type") != 0: continue
        for line in block.get("lines", []):
            for sp in line.get("spans", []):
                txt = (sp.get("text") or "").strip()
                if not txt: continue
                bbox = fitz.Rect(sp["bbox"])
                flags = int(sp.get("flags", 0))
                font = (sp.get("font") or "").lower()
                is_bold = ("bold" in font) or (flags & 16)
                spans.append({"text": txt, "bbox": bbox, "is_bold": bool(is_bold)})
    return spans


def build_lines_from_spans(spans):
    spans = sorted(spans, key=lambda s: (round(s["bbox"].y0, 1), s["bbox"].x0))
    lines = []
    for sp in spans:
        r = sp["bbox"]
        placed = False
        for ln in lines:
            if abs(ln["_y0"] - r.y0) <= SAME_LINE_Y:
                ln["spans"].append(sp)
                ln["_y0"] = (ln["_y0"] + r.y0) / 2.0
                ln["_bbox"] = union_rect(ln["_bbox"], r)
                placed = True; break
        if not placed:
            lines.append({"_y0": r.y0, "_bbox": fitz.Rect(r), "spans": [sp]})
    out = []
    for ln in lines:
        sps = sorted(ln["spans"], key=lambda s: s["bbox"].x0)
        parts = []; prev = None
        for sp in sps:
            if prev is None:
                parts.append(sp["text"])
            else:
                gap = sp["bbox"].x0 - prev["bbox"].x1
                if gap > SAME_LINE_X_GAP:
                    parts.append(" " + sp["text"])
                elif parts and not parts[-1].endswith((" ", "-", "\u201c", "\"", "(", "/")) \
                     and not sp["text"].startswith((",", ".", ")", ":", ";")):
                    parts.append(" " + sp["text"])
                else:
                    parts.append(sp["text"])
            prev = sp
        text = "".join(parts).strip()
        bold_count = sum(1 for sp in sps if sp["is_bold"])
        out.append({"text": text, "bbox": ln["_bbox"], "spans": sps,
                     "is_bold_line": bold_count / max(1, len(sps)) >= 0.6})
    out.sort(key=lambda x: (x["bbox"].y0, x["bbox"].x0))
    return out


def merge_bold_lines(bold_lines):
    if not bold_lines: return []
    bold_lines = sorted(bold_lines, key=lambda x: (x["bbox"].y0, x["bbox"].x0))
    merged = []; cur = None
    for ln in bold_lines:
        if cur is None:
            cur = {"text": ln["text"], "bbox": fitz.Rect(ln["bbox"])}; continue
        dy = ln["bbox"].y0 - cur["bbox"].y1
        same = abs(ln["bbox"].y0 - cur["bbox"].y0) <= SAME_LINE_Y and ln["bbox"].x0 >= cur["bbox"].x0
        nxt = (0.0 <= dy <= Y_GAP_MERGE) and abs(ln["bbox"].x0 - cur["bbox"].x0) <= X_TOL_MERGE
        if same or nxt:
            cur["text"] = (cur["text"] + " " + ln["text"]).strip()
            cur["bbox"] = union_rect(cur["bbox"], ln["bbox"])
        else:
            merged.append(cur); cur = {"text": ln["text"], "bbox": fitz.Rect(ln["bbox"])}
    if cur: merged.append(cur)
    for m in merged: m["text"] = re.sub(r"\s{2,}", " ", m["text"]).strip()
    return merged


def detect_qstarts_layout(lines, exclude_rects, page_no):
    q = []
    for ln in lines:
        skip = False
        for er in exclude_rects:
            if intersects(ln["bbox"], er):
                skip = True; break
        if skip: continue
        m = RE_QSTART.match(ln["text"])
        if m:
            q.append({"num": int(m.group(1)), "bbox": ln["bbox"], "text": ln["text"]})
    q.sort(key=lambda x: (x["bbox"].y0, x["bbox"].x0))
    return q


def classify_bolds(merged_bolds, qstarts, exclude_rects, page_no, page_height, next_qstarts=None):
    chapters, hinges = [], []
    for b in merged_bolds:
        skip = False
        for er in exclude_rects:
            if intersects(b["bbox"], er):
                skip = True; break
        if skip: continue
        txt = b["text"].strip()
        if RE_ROMAN.match(txt):
            chapters.append({"type": "chapter", "page": page_no, "text": txt,
                             "bbox": [b["bbox"].x0, b["bbox"].y0, b["bbox"].x1, b["bbox"].y1],
                             "sort_key": (page_no, b["bbox"].y0)})
            continue
        b_bottom = b["bbox"].y1
        cand = None
        for qs in qstarts:
            if qs["bbox"].y0 < b_bottom: continue
            gap = qs["bbox"].y0 - b_bottom
            if gap <= MAX_BISAGRA_TO_Q_GAP: cand = qs; break
            if gap > MAX_BISAGRA_TO_Q_GAP: break
        if cand is None and next_qstarts and (page_height - b_bottom) <= BOTTOM_PAGE_MARGIN:
            for qs in next_qstarts:
                if qs["bbox"].y0 <= TOP_NEXT_PAGE_SEARCH: cand = qs; break
        if cand is not None:
            hinges.append({"type": "hinge", "page": page_no, "text": txt,
                           "bbox": [b["bbox"].x0, b["bbox"].y0, b["bbox"].x1, b["bbox"].y1],
                           "sort_key": (page_no, b["bbox"].y0)})
    return chapters, hinges


def filter_questions_by_continuity(all_qs):
    if not all_qs: return []
    sorted_qs = sorted(all_qs, key=lambda q: q["sort_key"])
    filtered = []; last = -1
    for q in sorted_qs:
        if last < 0 or q["num"] >= last:
            filtered.append(q); last = q["num"]
    return filtered


def build_hierarchy(chapters, hinges, all_questions):
    timeline = []
    for ch in chapters: timeline.append({"kind": "chapter", "sort_key": ch["sort_key"], "data": ch})
    for h in hinges:   timeline.append({"kind": "hinge",   "sort_key": h["sort_key"],  "data": h})
    for q in all_questions: timeline.append({"kind": "question", "sort_key": q["sort_key"], "data": q})
    timeline.sort(key=lambda x: x["sort_key"])

    result = []; cur_ch = None; cur_h = None

    def fin_h():
        nonlocal cur_h
        if cur_h and cur_ch: cur_ch["hinges"].append(cur_h)
        cur_h = None

    def fin_ch():
        nonlocal cur_ch, cur_h
        fin_h()
        if cur_ch: result.append(cur_ch)
        cur_ch = None

    for item in timeline:
        k = item["kind"]
        if k == "chapter":
            fin_ch()
            d = item["data"]
            cur_ch = {"page": d["page"], "text": d["text"], "bbox": d["bbox"],
                      "questions": [], "hinges": [], "questions_without_hinge": []}
            cur_h = None
        elif k == "hinge":
            if not cur_ch: continue
            fin_h()
            d = item["data"]
            cur_h = {"page": d["page"], "text": d["text"], "bbox": d["bbox"], "questions": []}
        elif k == "question":
            qn = item["data"]["num"]
            if cur_ch:
                cur_ch["questions"].append(qn)
                if cur_h: cur_h["questions"].append(qn)
                else: cur_ch["questions_without_hinge"].append(qn)
    fin_ch()
    return result


def build_question_lookup(hierarchy):
    lookup = {}
    for ch in hierarchy:
        for h in ch["hinges"]:
            for qn in h["questions"]:
                lookup[qn] = {"capitulo": ch["text"], "bisagra": h["text"]}
        for qn in ch["questions_without_hinge"]:
            lookup[qn] = {"capitulo": ch["text"], "bisagra": None}
    return lookup


# #############################################################################
#  ASOCIAR TABLAS/FIGURAS → PREGUNTA (por posición)
# #############################################################################
def find_parent_question(sort_keys, qs_sorted, page, y0):
    idx = bisect_right(sort_keys, (page, y0)) - 1
    return qs_sorted[idx]["num"] if idx >= 0 else None


# #############################################################################
#  SALIDA
# #############################################################################
def save_outputs(out_dir, texto_total, preguntas_final, hierarchy):
    outp = Path(out_dir)
    outp.mkdir(parents=True, exist_ok=True)
    (outp / "texto_total.txt").write_text(texto_total, encoding="utf-8")
    (outp / "preguntas.json").write_text(json.dumps(preguntas_final, ensure_ascii=False, indent=2), encoding="utf-8")
    with (outp / "preguntas.txt").open("w", encoding="utf-8") as f:
        for p in preguntas_final:
            f.write(SEPARADOR_PREGUNTA + "\n")
            if p["capitulo"]: f.write(f"CAPITULO: {p['capitulo']}\n")
            if p["bisagra"]:  f.write(f"BISAGRA: {p['bisagra']}\n")
            f.write(f"NUMERO: {p['numero']}\n")
            f.write(p["texto"] + "\n")
            if p["tablas_figuras"]:
                for tf in p["tablas_figuras"]:
                    f.write(f"  [{tf['tipo'].upper()}] parte {tf['parte']}: {tf['png']}\n")
        f.write(SEPARADOR_PREGUNTA + "\n")
    (outp / "chapters_hinges.json").write_text(
        json.dumps({"chapters": hierarchy}, ensure_ascii=False, indent=2), encoding="utf-8")


# #############################################################################
#  MAIN
# #############################################################################
def main():
    if not PDF_PATH.exists():
        print(f"No se encontro el PDF: {PDF_PATH}")
        return

    doc = fitz.open(str(PDF_PATH))
    outp = Path(OUT_DIR)
    png_dir = outp / PNG_DIRNAME

    total_pages = len(doc)

    # =========================================================================
    # FASE 1: Detectar tablas/figuras + exportar PNGs + extraer texto sin tablas
    # =========================================================================
    all_detections = []           # lista global de detecciones
    pages_text_clean = []         # texto por página (sin tablas/figuras)
    all_page_layout_data = []     # datos de layout por página
    exclude_by_page = {}          # {page_no: [Rect, ...]}

    for pno in range(total_pages):
        page = doc[pno]
        page_no = pno + 1

        # Detectar bboxes a excluir
        tables = extract_table_candidates(page)
        figs = extract_raster_figures(page)
        excludes = tables + figs
        exclude_by_page[page_no] = excludes

        # Registrar detecciones (sin nombre aún, se asigna en Fase 2)
        for r in tables:
            all_detections.append({
                "tipo": "tabla", "page": page_no, "page_idx": pno,
                "bbox": r, "pregunta": None, "parte": None, "png": None,
            })
        for r in figs:
            all_detections.append({
                "tipo": "figura", "page": page_no, "page_idx": pno,
                "bbox": r, "pregunta": None, "parte": None, "png": None,
            })

        # Texto de la página excluyendo tablas/figuras
        page_text = extract_page_text_excluding_bboxes(page, excludes)
        pages_text_clean.append(page_text)

        # Layout data para Módulo B
        spans = extract_spans(page)
        lines = build_lines_from_spans(spans)
        bold_lines = [ln for ln in lines if ln["is_bold_line"] and ln["text"]]
        merged_bolds = merge_bold_lines(bold_lines)
        qstarts = detect_qstarts_layout(lines, excludes, page_no)

        all_page_layout_data.append({
            "page_no": page_no, "page_height": page.rect.height,
            "merged_bolds": merged_bolds, "qstarts": qstarts,
        })

    # =========================================================================
    # FASE 2: Texto plano → preguntas
    # =========================================================================
    pages_lines = [normalize_lines_keep_empty(t) for t in pages_text_clean]
    frequent_lines = build_frequent_line_filter(pages_lines)
    pages_clean_lines = [clean_page_lines_keep_empty(lines, frequent_lines) for lines in pages_lines]
    texto_total = stitch_pages(pages_clean_lines)
    preguntas_text = extract_questions_from_text(texto_total)

    # =========================================================================
    # FASE 3: Layout → capítulos, bisagras, jerarquía
    # =========================================================================
    all_chapters, all_hinges = [], []
    for i, pd in enumerate(all_page_layout_data):
        next_qs = all_page_layout_data[i + 1]["qstarts"] if i + 1 < len(all_page_layout_data) else None
        excludes = exclude_by_page.get(pd["page_no"], [])
        chs, hgs = classify_bolds(
            pd["merged_bolds"], pd["qstarts"], excludes,
            pd["page_no"], pd["page_height"], next_qstarts=next_qs)
        all_chapters.extend(chs)
        all_hinges.extend(hgs)

    all_qs_raw = []
    for pd in all_page_layout_data:
        for qs in pd["qstarts"]:
            all_qs_raw.append({"num": qs["num"], "page": pd["page_no"],
                               "sort_key": (pd["page_no"], qs["bbox"].y0)})
    all_qs_filtered = filter_questions_by_continuity(all_qs_raw)
    hierarchy = build_hierarchy(all_chapters, all_hinges, all_qs_filtered)
    lookup = build_question_lookup(hierarchy)

    # =========================================================================
    # FASE 4: Asociar detecciones → preguntas + exportar PNGs
    # =========================================================================
    sort_keys = [(q["page"], q["sort_key"][1]) for q in sorted(all_qs_filtered, key=lambda q: q["sort_key"])]
    qs_sorted = sorted(all_qs_filtered, key=lambda q: q["sort_key"])

    part_counters = defaultdict(int)

    for det in all_detections:
        parent_q = find_parent_question(sort_keys, qs_sorted, det["page"], det["bbox"].y0)
        det["pregunta"] = parent_q
        q_label = f"{parent_q:03d}" if parent_q is not None else "000"
        part_counters[(q_label, det["tipo"])] += 1
        parte = part_counters[(q_label, det["tipo"])]
        det["parte"] = parte

        fname = f"p{q_label}_parte{parte:03d}_{det['tipo']}.png"
        png_path = save_bbox_screenshot(doc, det["page_idx"], det["bbox"], png_dir, fname)
        det["png"] = fname  # solo nombre, no ruta completa

    doc.close()

    # =========================================================================
    # FASE 5: Cruzar preguntas + limpiezas + asociar tablas/figuras
    # =========================================================================
    all_hinge_texts = []
    for ch in hierarchy:
        for h in ch["hinges"]:
            all_hinge_texts.append(h["text"])

    # Agrupar detecciones por pregunta
    dets_by_q = defaultdict(list)
    for det in all_detections:
        if det["pregunta"] is not None:
            dets_by_q[det["pregunta"]].append({
                "tipo": det["tipo"], "parte": det["parte"], "png": det["png"],
            })

    preguntas_final = []
    for p in preguntas_text:
        num = p["numero"]
        info = lookup.get(num, {})
        texto = p["texto"]
        texto = clean_firma_digital(texto)
        texto = clean_trailing_hinge(texto, all_hinge_texts)

        tf_list = sorted(dets_by_q.get(num, []), key=lambda x: (x["tipo"], x["parte"]))

        preguntas_final.append({
            "capitulo": info.get("capitulo", ""),
            "bisagra": info.get("bisagra"),
            "numero": num,
            "texto": texto,
            "tablas_figuras": tf_list,
        })

    # =========================================================================
    # GUARDAR
    # =========================================================================
    save_outputs(outp, texto_total, preguntas_final, hierarchy)

    # =========================================================================
    # REPORTE
    # =========================================================================
    n_caps = len(hierarchy)
    n_bis = sum(len(ch["hinges"]) for ch in hierarchy)
    n_preg = len(preguntas_final)
    n_det = len(all_detections)
    n_tab = sum(1 for d in all_detections if d["tipo"] == "tabla")
    n_fig = sum(1 for d in all_detections if d["tipo"] == "figura")

    print(f"PDF: {PDF_PATH}")
    print(f"Páginas: {total_pages}")
    print(f"Capítulos: {n_caps} | Bisagras: {n_bis} | Preguntas: {n_preg}")
    print(f"Tablas: {n_tab} | Figuras: {n_fig} | Total detecciones: {n_det}")
    print(f"PNGs en: {png_dir}")
    print(f"Salida en: {outp}")


if __name__ == "__main__":
    main()

In [ ]:
"""Clasificador temático de preguntas ICSARA (SEIA Chile)

Este script lee un archivo preguntas.json y, para cada pregunta, calcula afinidad temática
contra una taxonomía (temas + keywords). El cálculo usa coincidencias de keywords en tres
zonas del texto: capítulo, bisagra y texto principal, asignando pesos distintos por zona.

Para cada pregunta:
- Normaliza los textos para robustecer coincidencias (minúsculas, espacios y una versión sin acentos).
- Evalúa cada tema: detecta keywords presentes y suma puntaje según dónde aparece cada keyword.
- Filtra temas bajo un umbral mínimo para reducir falsos positivos.
- Ordena los temas detectados por score descendente.
- Devuelve una clasificación estructurada con:
  - tema_principal / tema_principal_id (tema con mayor score)
  - temas_principales / temas_principales_id (conjunto de temas que cumplen criterio de principal)
  - temas_secundarios (los siguientes temas más relevantes)
  - keywords_match (evidencia: keywords encontradas)
  - detalle por zona (capítulo/bisagra/texto) para trazabilidad

Finalmente exporta preguntas_clasificadas.json con los campos originales más las etiquetas,
scores y evidencias de coincidencia.

Uso:
  python clasificar_preguntas.py
"""

import os
import json
import re
from pathlib import Path
from collections import Counter

# =============================================================================
# CONFIG
# =============================================================================
BASE_DIR = Path(os.getenv("ICSARA_BASE_DIR", str(Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd())))
DEFAULT_OUT_DIR = Path(os.getenv("ICSARA_OUT_DIR", str(BASE_DIR / "salida_icsara")))
INPUT_JSON = Path(os.getenv("ICSARA_INPUT_JSON", str(DEFAULT_OUT_DIR / "preguntas.json")))
OUTPUT_JSON = Path(os.getenv("ICSARA_OUTPUT_JSON", str(DEFAULT_OUT_DIR / "preguntas_clasificadas.json")))
OUTPUT_JSON_DETALLE = Path(os.getenv("ICSARA_OUTPUT_JSON_DETALLE", str(DEFAULT_OUT_DIR / "preguntas_clasificadas_detalle.json")))

# Umbral mínimo de score para asignar un tema (evitar falsos positivos)
MIN_SCORE = 2

# Peso por zona de coincidencia
PESO_CAPITULO = 3.0
PESO_BISAGRA = 5.0
PESO_TEXTO = 1.0

# Regla de multi-principal:
# Todo tema con score >= max(MIN_SCORE, MULTI_PRINCIPAL_RATIO * top_score) se marca como principal
MULTI_PRINCIPAL_RATIO = 0.80

# =============================================================================
# TAXONOMÍA ICSARA — DICCIONARIO COMPLETO (con 2 temas nuevos)
# =============================================================================
TAXONOMIA = {
    "CALIDAD_AIRE": {
        "nombre": "Calidad del Aire y Emisiones Atmosféricas",
        "keywords": [
            "mp10", "mp2,5", "mp2.5", "pm10", "pm2.5", "pm2,5",
            "material particulado", "so2", "so₂", "dióxido de azufre",
            "no2", "no₂", "nox", "dióxido de nitrógeno", "óxidos de nitrógeno",
            "monóxido de carbono", "ozono troposférico", "plomo atmosférico",
            "benceno", "compuestos orgánicos volátiles", "cov",
            "emisiones atmosféricas", "emisiones fugitivas",
            "calpuff", "aermod", "screen3", "wrf", "calmet",
            "modelación de dispersión", "modelo de dispersión",
            "isopletas", "rosa de vientos", "estabilidad atmosférica",
            "fuente fija", "fuente difusa", "fuente móvil", "chimenea",
            "factor de emisión", "ap-42", "cems",
            "inventario de emisiones",
            "ds 12/2022", "ds 12/2010", "ds 104/2018", "ds 114/2002",
            "ds 115/2002", "ds 112/2002", "ds 136/2000", "ds 05/2023",
            "ds 13/2011", "ds 28/2013", "ds 29/2013", "ds 4/1992",
            "ds 37/2013", "ds 09/2023", "ds 138/2005", "ds 144/1961",
            "norma de calidad del aire", "norma de emisión atmosférica",
            "zona latente", "zona saturada", "ppda",
            "plan de prevención y descontaminación",
            "sinca", "calidad del aire",
            "percentil 98", "concentración 24h", "promedio anual",
            "estación de monitoreo de calidad del aire",
        ],
    },

    "RUIDO_VIBRACIONES": {
        "nombre": "Ruido y Vibraciones",
        "keywords": [
            "ruido", "vibraciones", "vibración",
            "nivel de presión sonora", "nps", "npseq", "npc",
            "decibel", "db(a)", "dba",
            "horario diurno", "horario nocturno",
            "ruido de fondo", "ruido residual",
            "barrera acústica", "pantalla acústica",
            "propagación sonora", "modelo acústico",
            "ruido tonal", "ruido impulsivo",
            "ruido submarino", "efecto corona",
            "ds 38/2011", "ds 38", "ds 146/1997", "ds 146",
            "ne-01", "ne-02",
            "zona i", "zona ii", "zona iii", "zona iv",
            "nch 1619",
            "predicción y evaluación de impactos por ruido",
        ],
    },

    "GEOLOGIA_SUELOS": {
        "nombre": "Geología, Geomorfología y Suelos",
        "keywords": [
            "geología", "geomorfología", "suelo", "suelos",
            "capacidad de uso", "clase de uso",
            "taxonomía usda", "serie de suelos",
            "perfil edáfico", "horizontes del suelo",
            "textura del suelo", "erosión",
            "erosión hídrica", "erosión eólica", "cárcavas",
            "aptitud agrícola", "aptitud forestal", "aptitud ganadera",
            "remoción en masa", "deslizamiento", "subsidencia",
            "sismicidad", "falla geológica", "riesgo geológico",
            "ciren", "sernageomin",
            "dl 3.557", "dl 3557",
            "contaminación de suelos",
            "permeabilidad del suelo", "pedregosidad",
        ],
    },

    "RECURSO_HIDRICO": {
        "nombre": "Recurso Hídrico, Hidrología e Hidrogeología",
        "keywords": [
            "recurso hídrico", "recursos hídricos",
            "hidrología", "hidrogeología", "hidrogeológico",
            "caudal ecológico", "caudal mínimo ecológico", "caudal ambiental",
            "derechos de aprovechamiento de aguas",
            "balance hídrico", "cuenca hidrográfica",
            "acuífero", "zona de recarga", "zona de descarga",
            "nivel freático", "piezometría", "conductividad hidráulica",
            "aguas subterráneas", "aguas superficiales",
            "calidad de aguas", "cuerpo receptor",
            "riles", "ptas", "planta de tratamiento de aguas",
            "dbo", "dqo", "sst", "nkt",
            "coliformes fecales", "coliformes totales",
            "curva de duración de caudales",
            "modflow", "modelo hidrogeológico",
            "nch 1333", "nch 409",
            "ds 90/2000", "ds 90", "ds 46/2002", "ds 46",
            "código de aguas", "dfl 1.122",
            "norma secundaria de calidad de aguas",
            "dga", "dirección general de aguas",
            "doh", "dirección de obras hidráulicas",
            "siss",
        ],
    },

    "GLACIARES": {
        "nombre": "Glaciares y Criósfera",
        "keywords": [
            "glaciar", "glaciares", "glaciaretes",
            "glaciar rocoso", "glaciares rocosos",
            "criósfera", "permafrost",
            "balance de masa glaciar", "retroceso glaciar",
            "zona periglaciar",
            "inventario de glaciares",
            "unidad de glaciología",
            "estrategia nacional de glaciares",
        ],
    },

    "FLORA_VEGETACION": {
        "nombre": "Flora y Vegetación",
        "keywords": [
            "flora", "vegetación", "vegetal",
            "inventario florístico", "catastro vegetacional",
            "formaciones vegetacionales", "formaciones xerofíticas",
            "unidades homogéneas de vegetación", "uhv",
            "cobertura vegetal",
            "flora leñosa", "flora no leñosa", "suculentas",
            "matorral", "bosque esclerófilo", "bosque nativo",
            "bosque de preservación",
            "categoría de conservación", "rce",
            "especie en categoría", "clasificación de especies",
            "monumento natural", "formación relictual",
            "fotointerpretación",
            "singularidades ambientales",
            "revegetación", "reforestación", "restauración ecológica",
            "ley 20.283", "bosque nativo",
            "ds 68/2009", "dl 701", "ds 4.363",
            "pas 148", "pas 149", "pas 150", "pas 151",
            "pas 152", "pas 153",
            "artículo 148", "artículo 149", "artículo 150",
            "artículo 151", "artículo 152", "artículo 153",
            "pas 127", "pas 128", "pas 129",
            "conaf",
        ],
    },

    "FAUNA": {
        "nombre": "Fauna Terrestre",
        "keywords": [
            "fauna", "fauna silvestre", "fauna terrestre",
            "ensamble faunístico",
            "categoría de conservación fauna",
            "herpetofauna", "mastofauna", "avifauna", "entomofauna",
            "aves", "mamíferos", "reptiles", "anfibios",
            "murciélagos", "quirópteros",
            "trampa sherman", "trampa tomahawk", "trampa de foso", "pitfall",
            "cámara trampa", "fotomonitoreo",
            "red de niebla", "redes de niebla",
            "transecto lineal", "punto de conteo",
            "censo visual", "censo auditivo",
            "rescate y relocalización", "relocalización de fauna",
            "corredor biológico", "endemismo",
            "plan de manejo de fauna", "perturbación controlada",
            "nidificación", "migración", "reproducción fauna",
            "riqueza de especies", "diversidad shannon", "diversidad simpson",
            "ley 19.473", "ley de caza", "ds 5/1998", "cites",
            "pas 146", "pas 147", "pas 123", "pas 124",
            "artículo 146", "artículo 147",
            "sag",
        ],
    },

    "ECOSISTEMAS_ACUATICOS": {
        "nombre": "Ecosistemas Acuáticos Continentales y Humedales",
        "keywords": [
            "ecosistema acuático", "ecosistemas acuáticos",
            "limnología", "limnológico",
            "fauna íctica", "ictiofauna", "peces continentales",
            "macroinvertebrados bentónicos", "macroinvertebrados",
            "fitoplancton", "zooplancton", "macrófitas",
            "índice biótico", "ibf", "ept", "bmwp",
            "hábitat acuático", "ribereño",
            "humedal", "humedales", "bofedal", "bofedales",
            "vega", "vegas", "turbera", "turberas",
            "humedal urbano", "humedales urbanos",
            "inventario nacional de humedales", "ramsar",
            "ley 21.202",
            "ds 15 mma",
            "pas 155", "pas 156", "pas 157", "pas 158", "pas 159",
            "artículo 155", "artículo 156", "artículo 157",
            "artículo 158", "artículo 159",
            "modificación de cauce", "modificaciones de cauce",
            "obra hidráulica", "obras hidráulicas",
            "regularización de cauce", "defensa de cauce",
            "extracción de ripio", "extracción de arena",
        ],
    },

    "ECOSISTEMAS_MARINOS": {
        "nombre": "Ecosistemas Marinos",
        "keywords": [
            "ecosistema marino", "ecosistemas marinos", "medio marino",
            "columna de agua", "sedimentos marinos",
            "biota marina", "macroalgas", "macrofauna bentónica",
            "intermareal", "submareal", "infralitoral",
            "batimetría", "corrientes marinas",
            "amerb", "concesión de acuicultura", "concesión marítima",
            "caleta", "pescadores artesanales",
            "biodiversidad marina", "borde costero",
            "ley 18.892", "lgpa",
            "ds 1/1992",
            "pas 111", "pas 112", "pas 113", "pas 114", "pas 115",
            "pas 116", "pas 117", "pas 118", "pas 119",
            "artículo 111", "artículo 112", "artículo 113",
            "artículo 114", "artículo 115", "artículo 116",
            "subpesca", "sernapesca", "directemar", "shoa",
        ],
    },

    "PATRIMONIO_CULTURAL": {
        "nombre": "Patrimonio Cultural, Arqueología y Paleontología",
        "keywords": [
            "patrimonio cultural", "patrimonio arqueológico",
            "arqueología", "arqueológico", "arqueológica",
            "paleontología", "paleontológico", "paleontológica",
            "monumento histórico", "monumento arqueológico",
            "monumento paleontológico", "monumento público",
            "zona típica", "zonas típicas",
            "santuario de la naturaleza",
            "consejo de monumentos nacionales", "cmn",
            "prospección superficial", "pozo de sondeo",
            "excavación arqueológica", "rescate arqueológico",
            "monitoreo arqueológico", "hallazgo fortuito",
            "artículo 26 ley 17.288",
            "conformidad cmn",
            "patrimonio tangible", "patrimonio intangible",
            "ley 17.288", "ds 484/1990", "ley 21.600",
            "pas 131", "pas 132", "pas 133", "pas 120",
            "artículo 131", "artículo 132", "artículo 133",
            "artículo 120",
        ],
    },

    "PAISAJE": {
        "nombre": "Paisaje y Valor Turístico",
        "keywords": [
            "paisaje", "paisajístico", "valor paisajístico",
            "calidad visual", "fragilidad visual",
            "cuenca visual", "cuencas visuales",
            "unidad de paisaje", "unidades de paisaje",
            "punto de observación", "puntos de observación",
            "carácter del paisaje",
            "intrusión visual", "obstrucción visual",
            "incompatibilidad visual", "artificialidad",
            "simulación visual", "fotomontaje",
            "valor turístico", "turístico",
            "zoit", "zona de interés turístico",
            "atractivo natural", "atractivo cultural",
            "sernatur",
        ],
    },

    "AREAS_PROTEGIDAS": {
        "nombre": "Áreas Protegidas y Sitios Prioritarios",
        "keywords": [
            "área protegida", "áreas protegidas",
            "snaspe", "parque nacional", "reserva nacional",
            "monumento natural",
            "santuario de la naturaleza",
            "sitio ramsar",
            "sitio prioritario", "sitios prioritarios",
            "amcp-mu", "parque marino",
            "bien nacional protegido",
            "acbpo", "área colocada bajo protección oficial",
            "objeto de protección", "objetos de protección",
            "registro nacional de áreas protegidas",
            "simbio", "sbap",
            "pas 120", "pas 121", "pas 130",
            "artículo 121",
        ],
    },

    "MEDIO_HUMANO": {
        "nombre": "Medio Humano",
        "keywords": [
            "medio humano",
            "dimensión geográfica", "dimensión demográfica",
            "dimensión antropológica", "dimensión socioeconómica",
            "dimensión de bienestar social",
            "grupos humanos", "grupo humano",
            "sistemas de vida y costumbres",
            "pueblo indígena", "pueblos indígenas",
            "ghppi", "comunidad indígena",
            "reasentamiento", "reasentamiento de comunidades",
            "alteración significativa sistemas de vida",
            "cargas ambientales",
            "convenio 169", "oit",
            "consulta indígena",
            "conadi",
            "percepción comunitaria",
            "territorio indígena",
            "actividades económicas locales",
        ],
    },

    "USO_TERRITORIO": {
        "nombre": "Uso del Territorio y Planificación Territorial",
        "keywords": [
            "uso del territorio", "planificación territorial",
            "instrumento de planificación territorial", "ipt",
            "plan regulador comunal", "prc",
            "plan regulador intercomunal", "pri",
            "plan regional de ordenamiento territorial", "prot",
            "zonificación", "uso de suelo",
            "compatibilidad territorial",
            "oguc", "ordenanza general",
            "subdivisión predial", "cambio de uso de suelo",
            "terreno rural", "límite urbano",
            "pas 160", "pas 161",
            "artículo 160", "artículo 161",
            "calificación de instalaciones industriales",
            "seremi minvu", "minvu",
        ],
    },

    "DESCRIPCION_PROYECTO": {
        "nombre": "Descripción del Proyecto",
        "keywords": [
            "descripción del proyecto",
            "partes y obras", "partes, obras y acciones",
            "fase de construcción", "fase de operación", "fase de cierre",
            "cronograma", "vida útil",
            "mano de obra", "monto de inversión",
            "suministros básicos", "insumos",
            "capacidad instalada", "tecnología",
            "layout", "planos", "coordenadas utm",
            "datum wgs84",
            "tipología de ingreso", "artículo 10",
            "modificación de proyecto", "artículo 12",
            "desarrollo por etapas", "artículo 14",
            "inicio de ejecución",
            "piscina de emergencia", "depósito de relave",
            "botadero de estériles", "planta de procesos",
        ],
    },

    # -------------------------
    # NUEVO: ÁREA DE INFLUENCIA
    # -------------------------
    "AREA_INFLUENCIA": {
        "nombre": "Área de Influencia",
        "keywords": [
            "área de influencia", "area de influencia",
            "delimitación del área de influencia", "delimitacion del area de influencia",
            "justificación del área de influencia", "justificacion del area de influencia",
            "criterios de delimitación", "criterios de delimitacion",
            "polígono de área de influencia", "poligono de area de influencia",
            "radio de influencia",
            "componente en el área de influencia", "componente en el area de influencia",
            "área de estudio", "area de estudio",
        ],
    },

    "RESIDUOS": {
        "nombre": "Residuos Sólidos y Peligrosos",
        "keywords": [
            "residuo", "residuos",
            "rsd", "residuos sólidos domiciliarios",
            "rsinp", "residuos sólidos industriales no peligrosos",
            "respel", "residuos peligrosos",
            "reas", "residuos de establecimientos de atención de salud",
            "riles", "residuos industriales líquidos",
            "plan de manejo de residuos", "manifiesto de declaración",
            "sidrep", "sinader",
            "disposición final", "valorización", "reciclaje",
            "relleno sanitario", "relleno de seguridad",
            "bodega respel",
            "toxicidad", "inflamabilidad", "corrosividad", "reactividad",
            "ds 148/2003", "ds 148",
            "ley 20.920", "ley rep",
            "ds 189/2005", "ds 594/1999", "ds 594",
            "pas 138", "pas 139", "pas 140", "pas 141",
            "pas 142", "pas 143", "pas 144", "pas 145",
            "pas 126",
            "artículo 138", "artículo 139", "artículo 140",
            "artículo 141", "artículo 142", "artículo 143",
            "artículo 144", "artículo 145",
        ],
    },

    "SUSTANCIAS_PELIGROSAS": {
        "nombre": "Sustancias Peligrosas",
        "keywords": [
            "sustancias peligrosas", "suspel",
            "hoja de datos de seguridad", "hds",
            "nch 382", "9 clases",
            "cubeto de contención", "contención secundaria",
            "compatibilidad química", "segregación",
            "bodega de sustancias peligrosas",
            "transporte de cargas peligrosas",
            "ds 43/2015", "ds 78/2009",
            "nch 2190", "nch 2120",
            "ds 298/1994",
        ],
    },

    "TRANSPORTE_VIALIDAD": {
        "nombre": "Transporte y Vialidad",
        "keywords": [
            "transporte", "vialidad",
            "impacto vial", "estudio de impacto vial",
            "imiv", "eistu",
            "generación de viajes", "atracción de viajes",
            "nivel de servicio", "capacidad vial",
            "flujo vehicular", "flujos vehiculares",
            "intersección", "intersecciones",
            "sectra", "uoct", "seremitt",
            "saturn", "transyt", "aimsun", "estraus",
            "vivaldi", "modem", "modec",
            "libre circulación", "conectividad",
            "tiempos de desplazamiento",
            "ley 20.958", "ds 30/2017",
            "ds 83/1985", "dfl 850",
            "ley de caminos",
            "dirección de vialidad",
        ],
    },

    "PLAN_MEDIDAS": {
        "nombre": "Plan de Medidas de Mitigación, Reparación y Compensación",
        "keywords": [
            "plan de medidas", "medida de mitigación",
            "medidas de mitigación", "medida de reparación",
            "medidas de reparación", "medida de compensación",
            "medidas de compensación",
            "compromiso ambiental voluntario",
            "compensación de biodiversidad",
            "plan de rescate", "plan de revegetación",
            "plan de reforestación",
            "equivalencia ecológica", "adicionalidad",
            "no pérdida neta", "permanencia",
            "artículo 97", "artículo 98", "artículo 99",
            "artículo 100", "artículo 101", "artículo 102",
        ],
    },

    "PLAN_CONTINGENCIAS": {
        "nombre": "Plan de Contingencias y Emergencias",
        "keywords": [
            "plan de contingencia", "plan de emergencia",
            "contingencias y emergencias",
            "prevención de contingencias",
            "riesgo ambiental", "riesgos ambientales",
            "derrame", "incendio", "explosión", "fuga",
            "protocolo de actuación", "simulacro",
            "sistema de alerta temprana",
            "matriz de riesgos",
            "artículo 103", "artículo 104",
        ],
    },

    "PLAN_SEGUIMIENTO": {
        "nombre": "Plan de Seguimiento de Variables Ambientales",
        "keywords": [
            "plan de seguimiento", "seguimiento ambiental",
            "monitoreo ambiental", "variable de seguimiento",
            "frecuencia de muestreo", "punto de monitoreo",
            "puntos de monitoreo",
            "umbrales de acción",
            "informe de seguimiento",
            "monitoreo participativo",
            "verificación de cumplimiento",
            "efectividad de medidas",
            "artículo 105",
        ],
    },

    "NORMATIVA_AMBIENTAL": {
        "nombre": "Legislación y Normativa Ambiental Aplicable",
        "keywords": [
            "normativa ambiental aplicable",
            "legislación ambiental",
            "norma de emisión", "normas de emisión",
            "norma de calidad", "normas de calidad",
            "norma primaria", "norma secundaria",
            "plan de cumplimiento",
            "norma de referencia", "normas de referencia",
        ],
    },

    "PARTICIPACION_CIUDADANA": {
        "nombre": "Participación Ciudadana y Consulta Indígena",
        "keywords": [
            "participación ciudadana", "pac",
            "observaciones ciudadanas",
            "respuesta a observaciones",
            "consulta indígena",
            "convenio 169", "oit",
            "participación ciudadana temprana",
            "ponderación de observaciones",
        ],
    },

    "CAMBIO_CLIMATICO": {
        "nombre": "Cambio Climático y Gases de Efecto Invernadero",
        "keywords": [
            "cambio climático",
            "gases de efecto invernadero", "gei",
            "co2", "co₂", "ch4", "ch₄", "n2o", "n₂o",
            "huella de carbono",
            "carbono negro", "forzantes climáticos",
            "adaptación al cambio climático",
            "vulnerabilidad climática",
            "escenario rcp", "escenario ssp",
            "ley 21.455", "ley marco cambio climático",
            "ndc chile",
        ],
    },

    "RIESGO_SALUD": {
        "nombre": "Riesgo para la Salud de la Población",
        "keywords": [
            "riesgo para la salud", "riesgo en salud",
            "evaluación de riesgo en salud",
            "vía de exposición", "vías de exposición",
            "inhalación", "ingestión", "contacto dérmico",
            "población susceptible", "poblaciones susceptibles",
            "radiación electromagnética",
            "vectores sanitarios",
            "epidemiología",
            "artículo 5 rseia",
        ],
    },

    "MINERIA": {
        "nombre": "Aspectos Mineros",
        "keywords": [
            "minería", "minero", "minera",
            "depósito de relave", "relaves", "relave",
            "relave en pasta", "relave espesado",
            "botadero de estériles", "estériles",
            "plan de cierre de faena", "cierre de faena minera",
            "drenaje ácido", "dam", "aguas ácidas",
            "mineral", "concentrado", "lixiviación",
            "chancado", "molienda", "flotación",
            "pila de lixiviación",
            "pas 135", "pas 136", "pas 137",
            "pas 121", "pas 122", "pas 125", "pas 154",
            "artículo 135", "artículo 136", "artículo 137",
            "sernageomin",
        ],
    },

    # -------------------------
    # NUEVO: PAS
    # -------------------------
    "PAS": {
        "nombre": "PAS (Permisos Ambientales Sectoriales)",
        "keywords": [
            "pas", "permiso ambiental sectorial", "permisos ambientales sectoriales",
            # Artículos (variantes)
            "artículo 111", "articulo 111", "art. 111",
            "artículo 112", "articulo 112", "art. 112",
            "artículo 113", "articulo 113", "art. 113",
            "artículo 114", "articulo 114", "art. 114",
            "artículo 115", "articulo 115", "art. 115",
            "artículo 116", "articulo 116", "art. 116",
            "artículo 117", "articulo 117", "art. 117",
            "artículo 118", "articulo 118", "art. 118",
            "artículo 119", "articulo 119", "art. 119",
            "artículo 120", "articulo 120", "art. 120",
            "artículo 121", "articulo 121", "art. 121",
            "artículo 122", "articulo 122", "art. 122",
            "artículo 123", "articulo 123", "art. 123",
            "artículo 124", "articulo 124", "art. 124",
            "artículo 125", "articulo 125", "art. 125",
            "artículo 126", "articulo 126", "art. 126",
            "artículo 127", "articulo 127", "art. 127",
            "artículo 128", "articulo 128", "art. 128",
            "artículo 129", "articulo 129", "art. 129",
            "artículo 130", "articulo 130", "art. 130",
            "artículo 131", "articulo 131", "art. 131",
            "artículo 132", "articulo 132", "art. 132",
            "artículo 133", "articulo 133", "art. 133",
            "artículo 134", "articulo 134", "art. 134",
            "artículo 135", "articulo 135", "art. 135",
            "artículo 136", "articulo 136", "art. 136",
            "artículo 137", "articulo 137", "art. 137",
            "artículo 138", "articulo 138", "art. 138",
            "artículo 139", "articulo 139", "art. 139",
            "artículo 140", "articulo 140", "art. 140",
            "artículo 141", "articulo 141", "art. 141",
            "artículo 142", "articulo 142", "art. 142",
            "artículo 143", "articulo 143", "art. 143",
            "artículo 144", "articulo 144", "art. 144",
            "artículo 145", "articulo 145", "art. 145",
            "artículo 146", "articulo 146", "art. 146",
            "artículo 147", "articulo 147", "art. 147",
            "artículo 148", "articulo 148", "art. 148",
            "artículo 149", "articulo 149", "art. 149",
            "artículo 150", "articulo 150", "art. 150",
            "artículo 151", "articulo 151", "art. 151",
            "artículo 152", "articulo 152", "art. 152",
            "artículo 153", "articulo 153", "art. 153",
            "artículo 154", "articulo 154", "art. 154",
            "artículo 155", "articulo 155", "art. 155",
            "artículo 156", "articulo 156", "art. 156",
            "artículo 157", "articulo 157", "art. 157",
            "artículo 158", "articulo 158", "art. 158",
            "artículo 159", "articulo 159", "art. 159",
            "artículo 160", "articulo 160", "art. 160",
            "artículo 161", "articulo 161", "art. 161",
            # PAS n
            "pas 111", "pas 112", "pas 113", "pas 114", "pas 115", "pas 116",
            "pas 117", "pas 118", "pas 119", "pas 120", "pas 121",
            "pas 122", "pas 123", "pas 124", "pas 125", "pas 126",
            "pas 127", "pas 128", "pas 129", "pas 130",
            "pas 131", "pas 132", "pas 133", "pas 134",
            "pas 135", "pas 136", "pas 137",
            "pas 138", "pas 139", "pas 140", "pas 141", "pas 142", "pas 143",
            "pas 144", "pas 145", "pas 146", "pas 147", "pas 148", "pas 149",
            "pas 150", "pas 151", "pas 152", "pas 153", "pas 154",
            "pas 155", "pas 156", "pas 157", "pas 158", "pas 159",
            "pas 160", "pas 161",
        ],
    },
}

# =============================================================================
# NORMALIZACIÓN DE TEXTO
# =============================================================================
def normalizar(texto: str) -> str:
    """Normaliza texto para matching: minúsculas, espacios."""
    if not texto:
        return ""
    t = texto.lower().strip()
    t = re.sub(r"\s+", " ", t)
    return t


def normalizar_sin_acentos(texto: str) -> str:
    """Normalización sin acentos para matching más flexible."""
    t = normalizar(texto)
    repl = {"á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "ü": "u", "ñ": "n"}
    for old, new in repl.items():
        t = t.replace(old, new)
    return t


# =============================================================================
# MOTOR DE CLASIFICACIÓN
# =============================================================================
def calcular_score_tema(
    tema_id: str,
    tema_data: dict,
    cap_norm: str,
    bis_norm: str,
    txt_norm: str,
    cap_sin: str,
    bis_sin: str,
    txt_sin: str,
) -> dict:
    """
    Calcula el score de un tema para una pregunta.
    Retorna: {"score": float, "matches": [str], "detalle": {...}}
    """
    keywords = tema_data["keywords"]
    matches = []
    score = 0.0
    detalle = {"capitulo": [], "bisagra": [], "texto": []}

    for kw in keywords:
        kw_norm = normalizar(kw)
        kw_sin = normalizar_sin_acentos(kw)

        # Keywords cortos: palabra completa
        if len(kw_norm) <= 4:
            pattern = r"\b" + re.escape(kw_norm) + r"\b"
            pattern_sin = r"\b" + re.escape(kw_sin) + r"\b"
        else:
            pattern = re.escape(kw_norm)
            pattern_sin = re.escape(kw_sin)

        found = False

        if cap_norm and (re.search(pattern, cap_norm) or re.search(pattern_sin, cap_sin)):
            score += PESO_CAPITULO
            detalle["capitulo"].append(kw)
            found = True

        if bis_norm and (re.search(pattern, bis_norm) or re.search(pattern_sin, bis_sin)):
            score += PESO_BISAGRA
            detalle["bisagra"].append(kw)
            found = True

        if txt_norm and (re.search(pattern, txt_norm) or re.search(pattern_sin, txt_sin)):
            score += PESO_TEXTO
            detalle["texto"].append(kw)
            found = True

        if found:
            matches.append(kw)

    return {"score": score, "matches": matches, "detalle": detalle}


def clasificar_pregunta(pregunta: dict) -> dict:
    """
    Clasifica una pregunta en temas ICSARA.
    Retorna:
      - tema_principal / tema_principal_id (compatibilidad: primer principal)
      - temas_principales / temas_principales_id (lista)
      - temas (lista completa ordenada con detalle)
    """
    cap = pregunta.get("capitulo", "") or ""
    bis = pregunta.get("bisagra", "") or ""
    txt = pregunta.get("texto", "") or ""

    cap_norm = normalizar(cap)
    bis_norm = normalizar(bis)
    txt_norm = normalizar(txt)

    cap_sin = normalizar_sin_acentos(cap)
    bis_sin = normalizar_sin_acentos(bis)
    txt_sin = normalizar_sin_acentos(txt)

    resultados = []
    for tema_id, tema_data in TAXONOMIA.items():
        r = calcular_score_tema(
            tema_id, tema_data,
            cap_norm, bis_norm, txt_norm,
            cap_sin, bis_sin, txt_sin,
        )
        if r["score"] >= MIN_SCORE:
            resultados.append({
                "id": tema_id,
                "nombre": tema_data["nombre"],
                "score": r["score"],
                "matches": r["matches"],
                "detalle": r["detalle"],
            })

    resultados.sort(key=lambda x: x["score"], reverse=True)

    if not resultados:
        return {
            "tema_principal": "Sin clasificar",
            "tema_principal_id": "SIN_CLASIFICAR",
            "temas_principales": [],
            "temas_principales_id": [],
            "temas": [],
        }

    top_score = resultados[0]["score"]
    thr_principal = max(MIN_SCORE, MULTI_PRINCIPAL_RATIO * top_score)

    principales = [t for t in resultados if t["score"] >= thr_principal]

    return {
        "tema_principal": principales[0]["nombre"],
        "tema_principal_id": principales[0]["id"],
        "temas_principales": [t["nombre"] for t in principales],
        "temas_principales_id": [t["id"] for t in principales],
        "temas": resultados,
    }


# =============================================================================
# MAIN
# =============================================================================
def main():
    input_path = INPUT_JSON
    if not input_path.exists():
        print(f"No se encontro: {input_path}")
        return

    preguntas = json.loads(input_path.read_text(encoding="utf-8"))
    print(f"Preguntas cargadas: {len(preguntas)}")
    print(f"Temas disponibles: {len(TAXONOMIA)}")
    print(f"Multi-principal ratio: {MULTI_PRINCIPAL_RATIO:.2f}")

    resultados_out = []
    resultados_detalle_out = []
    for p in preguntas:
        clf = clasificar_pregunta(p)

        # matches del tema top (para no saturar)
        top_matches = clf["temas"][0]["matches"][:10] if clf["temas"] else []

        # secundarios: top temas que NO quedaron en principales
        principales_set = set(clf.get("temas_principales", []))
        secundarios = [t for t in clf.get("temas", []) if t["nombre"] not in principales_set]

        resultados_out.append({
            "numero": p.get("numero"),
            "capitulo": p.get("capitulo", ""),
            "bisagra": p.get("bisagra"),
            "tema_principal": clf["tema_principal"],                 # compat
            "tema_principal_id": clf["tema_principal_id"],           # compat
            "temas_principales": clf.get("temas_principales", []),   # nuevo
            "temas_principales_id": clf.get("temas_principales_id", []),  # nuevo
            "score": clf["temas"][0]["score"] if clf["temas"] else 0,
            "temas_secundarios": [
                {"nombre": t["nombre"], "score": t["score"]}
                for t in secundarios[:3]
            ],
            "keywords_match": top_matches,
            "texto": p.get("texto", ""),
            "tablas_figuras": p.get("tablas_figuras", []),
        })

        resultados_detalle_out.append({
            "numero": p.get("numero"),
            "capitulo": p.get("capitulo", ""),
            "bisagra": p.get("bisagra"),
            "tema_principal": clf["tema_principal"],
            "tema_principal_id": clf["tema_principal_id"],
            "temas_principales": clf.get("temas_principales", []),
            "temas_principales_id": clf.get("temas_principales_id", []),
            "temas": clf.get("temas", []),
        })

    output_path = OUTPUT_JSON
    output_path.parent.mkdir(parents=True, exist_ok=True)
    output_path.write_text(
        json.dumps(resultados_out, ensure_ascii=False, indent=2),
        encoding="utf-8",
    )

    output_detalle_path = OUTPUT_JSON_DETALLE
    output_detalle_path.parent.mkdir(parents=True, exist_ok=True)
    output_detalle_path.write_text(
        json.dumps(resultados_detalle_out, ensure_ascii=False, indent=2),
        encoding="utf-8",
    )

    # === REPORTE ===
    print(f"\nSalida: {output_path}")
    print(f"Salida detalle: {output_detalle_path}")

    dist = Counter(r["tema_principal"] for r in resultados_out)
    n_sin = dist.get("Sin clasificar", 0)

    print(f"\nClasificadas: {len(resultados_out) - n_sin}/{len(resultados_out)}")
    if n_sin:
        print(f"Sin clasificar: {n_sin}")

    print(f"\nDistribución temática (por primer principal):")
    print("-" * 65)
    for tema, count in dist.most_common():
        bar = "#" * min(count, 40)
        print(f"  {tema[:45]:<45} {count:>3}  {bar}")

    print(f"\nEjemplos (primeras 10):")
    print("-" * 65)
    for r in resultados_out[:10]:
        prims = ", ".join(r.get("temas_principales", [])[:3])
        sec = ", ".join(t["nombre"][:30] for t in (r.get("temas_secundarios") or [])[:2])
        kws = ", ".join((r.get("keywords_match") or [])[:4])

        print(f"  P{r['numero']!s:>3}: {r['tema_principal'][:40]:<40} (score:{r['score']:.0f})")
        if prims:
            print(f"        Principales: {prims}")
        if sec:
            print(f"        Secundarios: {sec}")
        if kws:
            print(f"        Keywords: {kws}")

    sin_clf = [r["numero"] for r in resultados_out if r["tema_principal"] == "Sin clasificar"]
    if sin_clf:
        print(f"\nPreguntas sin clasificar: {sin_clf}")


if __name__ == "__main__":
    main()
